# Import Libraries

In [1]:
#Basic imports
import pandas as pd
import numpy as np
from tqdm import tqdm

#SkLearn general imports
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

#Keras imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras import optimizers

Using TensorFlow backend.


# Regression model

For the regression model, we are going to apply a Multilayer Perceptron network using the Keras library.

The first step is then to define the neural network as a function.

In [ ]:
#Define neural network as function
def mlp_regression(n_in, n_out, n_nodes, n_layers, act, loss_f, opt, drop):
  winit = 'glorot_uniform'  
    
  model = Sequential()
  model.add(Dense(units = n_nodes, activation = act, kernel_initializer = winit, input_dim = n_in))
  model.add(Dropout(drop))
  for i in range(n_layers):
    model.add(Dense(units = n_nodes, activation = act, kernel_initializer = winit))
    model.add(Dropout(drop))
  model.add(Dense(units = n_out, activation = 'linear', kernel_initializer = winit))
  
  model.compile(optimizer = opt, loss = loss_f, 
                metrics = ['mean_squared_logarithmic_error'])
  return model

In [ ]:
#Read datasets
train_data = pd.read_csv('train_final.csv')
test_data = pd.read_csv('test_final.csv')

In [ ]:
#Split train dataset into target and features
target = train_data['SalePrice'].values
atr = train_data.drop(columns = ['Id', 'SalePrice']).values

In [ ]:
#Scale train dataset
scaler = MinMaxScaler()
atr = scaler.fit_transform(atr)

We now perform a first run in order to have a preliminar score to serve as an anchor for the posterior optimizations.

In [ ]:
#Test model
first_model = KerasRegressor(build_fn = mlp_regression,
                             n_in = atr.shape[1],
                             n_out = 1,
                             n_nodes = 200,
                             n_layers = 1,
                             act = 'relu',
                             loss_f = 'mean_squared_logarithmic_error',
                             opt = 'adam',
                             drop = 0.2,
                             batch_size = 32,
                             epochs = 200)

score = cross_val_score(estimator = first_model,
                        X = atr,
                        y = target,
                        scoring = 'neg_mean_squared_log_error',
                        cv = 10,
                        n_jobs = 2,
                        verbose = 1)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  4.9min finished


In [ ]:
#Print first test mean score
-score.mean()

0.04353192985275292

This score is going to be taken as reference for our following optimization procedures.

## Tune Hyperparameters

Our tuning, at first, we'll be limited to the number of hidden layers and the number of neurons per layer.

In [ ]:
#Define hyperparameters grid
param = {'n_in': [atr.shape[1]],
         'n_out': [1],
         'n_layers': [1, 2, 3, 4],
         'n_nodes': [30, 50, 80, 100, 150, 200],
         'opt': ['adam'],
         'act': ['relu'],
         'loss_f': ['mean_squared_logarithmic_error'],
         'drop': [0.2],
         'batch_size': [32],
         'epochs': [500]}

In [ ]:
#Tune hyperparameters
model = KerasRegressor(build_fn = mlp_regression)
grid = GridSearchCV(estimator = model,
                    param_grid = param,
                    scoring = 'neg_mean_squared_log_error',
                    cv = 10,
                    n_jobs = 4,
                    verbose = 2)

grid.fit(atr, target)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 44.7min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 217.3min
[Parallel(n_jobs=4)]: Done 240 out of 240 | elapsed: 358.4min finished
W0903 03:16:23.227349 140210351200128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 03:16:23.249545 140210351200128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.comp

Epoch 1/500
1381/1381 [==============================] - 1s 906us/step - loss: 76.3630 - mean_squared_logarithmic_error: 76.3630
Epoch 2/500
1381/1381 [==============================] - 0s 195us/step - loss: 27.4164 - mean_squared_logarithmic_error: 27.4164
Epoch 3/500
1381/1381 [==============================] - 0s 185us/step - loss: 14.9743 - mean_squared_logarithmic_error: 14.9743
Epoch 4/500
1381/1381 [==============================] - 0s 162us/step - loss: 9.3384 - mean_squared_logarithmic_error: 9.3384
Epoch 5/500
1381/1381 [==============================] - 0s 174us/step - loss: 6.0254 - mean_squared_logarithmic_error: 6.0254
Epoch 6/500
1381/1381 [==============================] - 0s 173us/step - loss: 3.9596 - mean_squared_logarithmic_error: 3.9596
Epoch 7/500
1381/1381 [==============================] - 0s 189us/step - loss: 2.6158 - mean_squared_logarithmic_error: 2.6158
Epoch 8/500
1381/1381 [==============================] - 0s 181us/step - loss: 1.7710 - mean_squared_loga

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f84dcc25d30>,
             iid='warn', n_jobs=4,
             param_grid={'act': ['relu'], 'batch_size': [32], 'drop': [0.2],
                         'epochs': [500],
                         'loss_f': ['mean_squared_logarithmic_error'],
                         'n_in': [187], 'n_layers': [1, 2, 3, 4],
                         'n_nodes': [30, 50, 80, 100, 150, 200], 'n_out': [1],
                         'opt': ['adam']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_log_error', verbose=2)

In [ ]:
grid_csv[grid_csv['rank_test_score'] == 1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_act,param_batch_size,param_drop,param_epochs,param_loss_f,param_n_in,param_n_layers,param_n_nodes,param_n_out,param_opt,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
11,360.871571,11.30602,1.083509,0.864642,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,adam,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.016334,-0.010579,-0.0145,-0.015603,-0.022201,-0.00949,-0.015474,-0.011556,-0.01327,-0.018365,-0.014738,0.003603,1


The optimal hyperparameter values taken from the tuning are:

* Number of hidden layers: 2
* Number of neurons per layer: 200

The cross validation score for this set of parameters is 0.14738.

Now that we defined the best combination of number of hidden layers and neurons per layers, we turn to the other aspects of our neural network. First, we look at the optimizers.

In [ ]:
#Tune optimizers
param = {'n_in': [atr.shape[1]],
         'n_out': [1],
         'n_layers': [2],
         'n_nodes': [200],
         'opt': ['rmsprop', 'sgd', 'adam', 'adamax', 'nadam', 'adadelta', 'adagrad'],
         'act': ['relu'],
         'loss_f': ['mean_squared_logarithmic_error'],
         'drop': [0.2],
         'batch_size': [32],
         'epochs': [500]}

model = KerasRegressor(build_fn = mlp_regression)
grid = GridSearchCV(estimator = model,
                    param_grid = param,
                    scoring = 'neg_mean_squared_log_error',
                    cv = 10,
                    n_jobs = 2,
                    verbose = 2)

grid.fit(atr, target)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
#Print tuning results
pd.DataFrame(data = grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_act,param_batch_size,param_drop,param_epochs,param_loss_f,param_n_in,param_n_layers,param_n_nodes,param_n_out,param_opt,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,129.529746,3.247060,0.137599,0.047647,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,rmsprop,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.017119,-0.011274,-0.013467,-0.018049,-0.026194,-0.009585,-0.018149,-0.012026,-0.011558,-0.017449,-0.015488,0.004677,3
1,113.916474,0.965165,0.266155,0.040711,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,sgd,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.078768,-0.082557,-0.108098,-0.096943,-0.111388,-0.070290,-0.089717,-0.090619,-0.071183,-0.107616,-0.090709,0.014362,7
2,133.151514,2.329288,0.181102,0.115745,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,adam,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.019647,-0.011045,-0.014532,-0.018811,-0.024721,-0.011964,-0.019482,-0.012087,-0.012093,-0.016655,-0.016106,0.004281,5
3,126.581392,1.517086,0.280363,0.043146,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,adamax,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.018665,-0.010437,-0.013646,-0.016462,-0.024056,-0.011269,-0.020289,-0.011117,-0.011168,-0.017308,-0.015444,0.004418,2
4,151.372595,0.457846,0.174913,0.097857,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,nadam,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.017555,-0.011624,-0.014968,-0.017897,-0.025031,-0.012473,-0.017876,-0.011470,-0.010918,-0.018252,-0.015808,0.004180,4
5,143.424012,1.230157,0.297926,0.091323,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,adadelta,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.017387,-0.012883,-0.013440,-0.015768,-0.025380,-0.010268,-0.012817,-0.011771,-0.011848,-0.016295,-0.014788,0.004118,1
6,119.044850,1.549442,0.140117,0.056569,relu,32,0.2,500,mean_squared_logarithmic_error,187,2,200,1,adagrad,"{'act': 'relu', 'batch_size': 32, 'drop': 0.2,...",-0.026165,-0.024186,-0.026822,-0.034702,-0.041114,-0.020433,-0.026530,-0.021169,-0.020201,-0.033006,-0.027432,0.006516,6


Best optimizer: ADADELTA

Cross validation score: 0.14788

'Adadelta' is the optimizer that provides the best score in this step. Let us now look at the optimizer parameters. For this, we rewrite the neural network function.

In [ ]:
def mlp_regression(n_in, n_out, n_nodes, n_layers, act, loss_f, opt, opt_value, drop):
  winit = 'glorot_uniform'  
    
  model = Sequential()
  model.add(Dense(units = n_nodes, activation = act, kernel_initializer = winit, input_dim = n_in))
  model.add(Dropout(drop))
  for i in range(n_layers):
    model.add(Dense(units = n_nodes, activation = act, kernel_initializer = winit))
    model.add(Dropout(drop))
  model.add(Dense(units = n_out, activation = 'linear', kernel_initializer = winit))
  
  if opt == 0:
    opt = optimizers.Adadelta(clipnorm = opt_value)
  elif opt == 1:
    opt = optimizers.Adadelta(clipvalue = opt_value)
  model.compile(optimizer = opt, loss = loss_f, 
                metrics = ['mean_squared_logarithmic_error'])
  return model

In [ ]:
#Tune optimizers
param = {'n_in': [atr.shape[1]],
         'n_out': [1],
         'n_layers': [2],
         'n_nodes': [200],
         'opt': [0, 1],
         'opt_value': [0.5, 1.0, 2.0, 4.0],
         'act': ['relu'],
         'loss_f': ['mean_squared_logarithmic_error'],
         'drop': [0.2],
         'batch_size': [32],
         'epochs': [500]}

model = KerasRegressor(build_fn = mlp_regression)
grid = GridSearchCV(estimator = model,
                    param_grid = param,
                    scoring = 'neg_mean_squared_log_error',
                    cv = 10,
                    n_jobs = 4,
                    verbose = 2)

grid.fit(atr, target)

In [ ]:
#Print tuning results
pd.DataFrame(data = grid.cv_results_)

In this step, the set of tuned hyperparemeters that provide the best score are:

* opt: 1
* opt_value: 4.0

## Final Model

Finally, we apply the tuning results into our final model.

In [ ]:
mfinal = mlp_regression(n_in = atr.shape[1],
                        n_out = 1,
                        n_nodes = 200,
                        n_layers = 2,
                        act = 'relu',
                        loss_f = 'mean_squared_logarithmic_error',
                        opt = 1,
                        opt_value = 4.0,
                        drop = 0.2)

mfinal.fit(atr, target, batch_size = 16, epochs = 1000)                    

Epoch 1/1000
1381/1381 [==============================] - 1s 920us/step - loss: 37.8071 - mean_squared_logarithmic_error: 37.8071
Epoch 2/1000
1381/1381 [==============================] - 1s 370us/step - loss: 5.4905 - mean_squared_logarithmic_error: 5.4905
Epoch 3/1000
1381/1381 [==============================] - 1s 370us/step - loss: 1.5373 - mean_squared_logarithmic_error: 1.5373
Epoch 4/1000
1381/1381 [==============================] - 1s 377us/step - loss: 0.4151 - mean_squared_logarithmic_error: 0.4151
Epoch 5/1000
1381/1381 [==============================] - 1s 373us/step - loss: 0.1537 - mean_squared_logarithmic_error: 0.1537
Epoch 6/1000
1381/1381 [==============================] - 1s 388us/step - loss: 0.1092 - mean_squared_logarithmic_error: 0.1092
Epoch 7/1000
1381/1381 [==============================] - 1s 380us/step - loss: 0.1059 - mean_squared_logarithmic_error: 0.1059
Epoch 8/1000
1381/1381 [==============================] - 1s 400us/step - loss: 0.1042 - mean_squared_

## Test Model

In [ ]:
#Scale test data
x_test = scaler.transform(test_data.drop(columns = ['Id', 'SalePrice']).values)

In [ ]:
#Predict values
pred = mfinal.predict(x_test)
pred = [x[0] for x in pred]

In [ ]:
#Write prediction to file
submission = pd.Series(data = pred, index = test_data['Id'])
submission.to_csv('submission.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
